## 示例：调用天气API

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

# import os
# os.environ['OPENAI_API_KEY'] = 'sk-123456789'

# import os
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

True

In [2]:
from openai import OpenAI

GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI()

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "查询即时天气函数，根据输入的城市名称，查询对应城市的实时天气，一次只能输入一个城市名称",
            "parameters": {
                "type": "object",
                "properties": {
                    "loc": {
                        "description": "城市名称，注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'",
                        "type": "string"
                    }
                },
                "required": ["loc"]
            }
        }
    }
]

In [4]:
import requests
import json
import os

# 这里需要替换为你自己的OpenWeatherMap API密钥
open_weather_key = os.getenv("OPEN_WEATHER_API_KEY")

def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，
                注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return: OpenWeather API查询即时天气的结果，具体URL请求地址为: https://api.openweathermap.org/data/2.5/weather
             返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"
    # Step 2.设置查询参数
    params = {
        "q": loc,
        "appid": open_weather_key,  # 输入API key
        "units": "metric",  # 使用摄氏度而不是华氏度
        "lang": "zh_cn"  # 输出语言为简体中文
    }
    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [5]:
response = client.chat.completions.create(
    model=GPT_MODEL,
    messages=[
        {"role": "user", "content": "你好！"}
    ],
    tools=tools
)

print(response.choices[0].message.content)

你好！有什么可以帮助你的吗？


In [8]:
# 用户提问
messages = [
    {"role": "user", "content": "请问北京今天天气如何？"}
]
response = client.chat.completions.create(
    model=GPT_MODEL,
    messages=messages,
    tools=tools
)

# 获取工具调用信息
function_call_message = response.choices[0].message.tool_calls
if function_call_message:
    print(function_call_message)
    function_call = function_call_message[0]
    if function_call.function.name == "get_weather":
        arguments = json.loads(function_call.function.arguments)
        loc = arguments["loc"]
        weather_result = get_weather(loc)
        print(weather_result)
        # 追加function返回消息
        messages.append({
            "role": "tool",
            "content": weather_result,
            "tool_call_id": response.choices[0].message.tool_calls[0].id
        })

[ChatCompletionMessageToolCall(id='call_nCUhGtiRPiK2ETobrY1gv9z4', function=Function(arguments='{"loc":"Beijing"}', name='get_weather'), type='function')]
{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 803, "main": "Clouds", "description": "\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 17.94, "feels_like": 16.7, "temp_min": 17.94, "temp_max": 17.94, "pressure": 1013, "humidity": 35, "sea_level": 1013, "grnd_level": 1008}, "visibility": 10000, "wind": {"speed": 3.42, "deg": 201, "gust": 5.69}, "clouds": {"all": 60}, "dt": 1745116514, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1745098176, "sunset": 1745146604}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}


In [9]:
# 再次请求模型
second_response = client.chat.completions.create(
    model=GPT_MODEL,
    messages=messages,
    tools=tools
)

# 打印最终回复
print(second_response.choices[0].message.content)

根据提供的天气数据，北京今天天气状况为多云，温度为17.94摄氏度，体感温度为16.7摄氏度。湿度为35%，风速为3.42米/秒，风向为201度，阵风风速为5.69米/秒。能见度为10000米。云量为60%。


总的来说，北京今天天气较为温和，多云，体感略微凉爽。

